In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [4]:
# Load the data
file_path = Path("Machine Learning CSVs/encoded_loans_df.csv")
df = pd.read_csv(file_path, index_col="LoanNumber")
df

,BorrowerZip,LoanStatus,Term,InitialApprovalAmount,CurrentApprovalAmount,JobsReported,ForgivenessAmount,rating,review_count,lat,...,MonthApproved_1,MonthApproved_2,MonthApproved_3,MonthApproved_4,MonthApproved_5,MonthApproved_6,MonthApproved_7,MonthApproved_8,YearApproved_2020,YearApproved_2021
LoanNumber,,,,,,,,,,,,,,,,,,,,,
8728957203,80202,Paid in Full,24,7275742.0,7275742.0,500.0,7356987.79,1.5,7,39.754760,...,0,0,0,1,0,0,0,0,1,0
9826177105,80204,Paid in Full,24,5865500.0,5865500.0,391.0,5943869.60,2.5,3,39.737240,...,0,0,0,1,0,0,0,0,1,0
2089597208,80202,Paid in Full,24,5114560.0,5114560.0,227.0,5173803.65,3.0,2,39.751680,...,0,0,0,1,0,0,0,0,1,0
8574487102,80230,Paid in Full,24,3527000.0,3527000.0,249.0,3573732.75,1.0,4,39.717893,...,0,0,0,1,0,0,0,0,1,0
7218167001,80216,Exemption 4,60,3230500.0,3230500.0,500.0,971592.57,1.0,1,39.771974,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7810438506,80222,Exemption 4,60,1327.0,1327.0,1.0,NaN,5.0,9,39.675860,...,0,0,1,0,0,0,0,0,0,1
2773518503,80237,Paid in Full,60,1000.0,1000.0,1.0,1009.84,5.0,5,39.598190,...,0,1,0,0,0,0,0,0,0,1
4250667310,80237,Paid in Full,24,1000.0,1000.0,1.0,1008.25,5.0,5,39.598190,...,0,0,0,1,0,0,0,0,1,0


In [5]:
# drop ForgivenessAmount column
df = df.drop('ForgivenessAmount', axis  = 1 )
df = df.drop('CurrentApprovalAmount', axis = 1)
df

,BorrowerZip,LoanStatus,Term,InitialApprovalAmount,JobsReported,rating,review_count,lat,lon,BusinessAgeDescription_Existing or more than 2 years old,...,MonthApproved_1,MonthApproved_2,MonthApproved_3,MonthApproved_4,MonthApproved_5,MonthApproved_6,MonthApproved_7,MonthApproved_8,YearApproved_2020,YearApproved_2021
LoanNumber,,,,,,,,,,,,,,,,,,,,,
8728957203,80202,Paid in Full,24,7275742.0,500.0,1.5,7,39.754760,-104.997910,1,...,0,0,0,1,0,0,0,0,1,0
9826177105,80204,Paid in Full,24,5865500.0,391.0,2.5,3,39.737240,-105.008310,1,...,0,0,0,1,0,0,0,0,1,0
2089597208,80202,Paid in Full,24,5114560.0,227.0,3.0,2,39.751680,-104.998950,1,...,0,0,0,1,0,0,0,0,1,0
8574487102,80230,Paid in Full,24,3527000.0,249.0,1.0,4,39.717893,-104.895233,1,...,0,0,0,1,0,0,0,0,1,0
7218167001,80216,Exemption 4,60,3230500.0,500.0,1.0,1,39.771974,-104.995255,1,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7810438506,80222,Exemption 4,60,1327.0,1.0,5.0,9,39.675860,-104.922090,1,...,0,0,1,0,0,0,0,0,0,1
2773518503,80237,Paid in Full,60,1000.0,1.0,5.0,5,39.598190,-104.903560,1,...,0,1,0,0,0,0,0,0,0,1
4250667310,80237,Paid in Full,24,1000.0,1.0,5.0,5,39.598190,-104.903560,1,...,0,0,0,1,0,0,0,0,1,0


In [6]:
# Create our features
X = df.drop("LoanStatus", axis=1)

X= pd.get_dummies(X)

# Create our target
y = df["LoanStatus"]
print(y)

LoanNumber
8728957203    Paid in Full
9826177105    Paid in Full
2089597208    Paid in Full
8574487102    Paid in Full
7218167001     Exemption 4
                  ...     
7810438506     Exemption 4
2773518503    Paid in Full
4250667310    Paid in Full
5634797307    Paid in Full
3616317801    Paid in Full
Name: LoanStatus, Length: 1294, dtype: object


In [7]:
X.describe()

,BorrowerZip,Term,InitialApprovalAmount,JobsReported,rating,review_count,lat,lon,BusinessAgeDescription_Existing or more than 2 years old,BusinessAgeDescription_New Business or 2 years or less,...,MonthApproved_1,MonthApproved_2,MonthApproved_3,MonthApproved_4,MonthApproved_5,MonthApproved_6,MonthApproved_7,MonthApproved_8,YearApproved_2020,YearApproved_2021
count,1294.000000,1294.000000,1.294000e+03,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000,...,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000,1294.000000
mean,80215.329212,40.029366,1.363728e+05,14.243431,4.127512,71.820711,39.722119,-104.967353,0.929675,0.070325,...,0.098145,0.170015,0.072643,0.428903,0.194745,0.017774,0.011592,0.006182,0.615920,0.384080
std,18.767428,17.940110,4.051040e+05,36.404919,1.065330,177.310247,0.047305,0.052492,0.255792,0.255792,...,0.297626,0.375792,0.259650,0.495111,0.396157,0.132181,0.107082,0.078415,0.486565,0.486565
min,80012.000000,2.000000,5.000000e+02,1.000000,1.000000,1.000000,39.368946,-105.245930,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,80205.000000,24.000000,1.765275e+04,2.000000,4.000000,3.000000,39.692500,-104.995303,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,80211.000000,24.000000,4.260943e+04,5.000000,4.500000,9.000000,39.727896,-104.976669,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,80222.000000,60.000000,1.073250e+05,12.000000,5.000000,52.000000,39.751432,-104.937035,1.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,80401.000000,60.000000,7.275742e+06,500.000000,5.000000,1661.000000,40.036910,-104.670690,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
# Check the balance of our target values
y.value_counts()

Paid in Full    1176
Exemption 4      118
Name: LoanStatus, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=1)

### Balanced Random Forest Classifier

In [10]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
random_forest = BalancedRandomForestClassifier(n_estimators = 100)
random_forest = random_forest.fit(X_train, y_train)

In [11]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = random_forest.predict(X_test)
print("Balanced Accuracy")
balanced_accuracy_score(y_test, y_pred)

Balanced Accuracy


0.7683673469387755

In [12]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual Exemption 4", "Actual Paid in Full"], columns=["Predicted Exemption 4", "Predicted Paid in Full"]
)
# Displaying results
print("Confusion Matrix")
display(cm_df)

Confusion Matrix


,Predicted Exemption 4,Predicted Paid in Full
Actual Exemption 4,26,4
Actual Paid in Full,97,197


In [13]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                    pre       rec       spe        f1       geo       iba       sup

 Exemption 4       0.21      0.87      0.67      0.34      0.76      0.59        30
Paid in Full       0.98      0.67      0.87      0.80      0.76      0.57       294

 avg / total       0.91      0.69      0.85      0.75      0.76      0.57       324



In [14]:
# List the features sorted in descending order by feature importance
feature_names = X.columns
sorted(zip(random_forest.feature_importances_, feature_names), reverse=True)

[(0.1185391816129899, 'InitialApprovalAmount'),
 (0.08676368500084301, 'lon'),
 (0.08539923432233391, 'lat'),
 (0.07708549274013116, 'review_count'),
 (0.07369921365576813, 'Term'),
 (0.06984986278432562, 'YearApproved_2021'),
 (0.06835021633923989, 'BorrowerZip'),
 (0.05871483777806223, 'JobsReported'),
 (0.05806067888781556, 'rating'),
 (0.04733112131819858, 'YearApproved_2020'),
 (0.03516436494930075, 'MonthApproved_4'),
 (0.018197446548058428, 'Yelp_Category_Health'),
 (0.01693260590553856, 'Yelp_Category_Restaurants'),
 (0.016772900994272356, 'MonthApproved_2'),
 (0.01643784139212431, 'MonthApproved_5'),
 (0.015586057123526948, 'BusinessType_Corporation'),
 (0.01467045781774202, 'BusinessType_Limited  Liability Company(LLC)'),
 (0.01442296736463523, 'Yelp_Category_Professional Services'),
 (0.013380096049669533, 'MonthApproved_1'),
 (0.012433025475117753, 'Yelp_Category_Hair Salon'),
 (0.012086164962780659, 'BusinessType_Subchapter S Corporation'),
 (0.009644867223026087, 'MonthAp

### Easy Ensemble AdaBoost Classifier

In [15]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
easy = EasyEnsembleClassifier(n_estimators = 100,random_state=1)
easy = easy.fit(X_train, y_train)

In [16]:
# Calculated the balanced accuracy score
y_pred = easy.predict(X_test)
print("Balanced Accuracy")
balanced_accuracy_score(y_test, y_pred)


Balanced Accuracy


0.689795918367347

In [17]:
# Display the confusion matrix
# Calculating the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(
    cm, index=["Actual Exemption 4", "Actual Paid in Full"], columns=["Predicted Exemption 4", "Predicted Paid in Full"]
)
# Displaying results
print("Confusion Matrix")
display(cm_df)


Confusion Matrix


,Predicted Exemption 4,Predicted Paid in Full
Actual Exemption 4,22,8
Actual Paid in Full,104,190


In [18]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                    pre       rec       spe        f1       geo       iba       sup

 Exemption 4       0.17      0.73      0.65      0.28      0.69      0.48        30
Paid in Full       0.96      0.65      0.73      0.77      0.69      0.47       294

 avg / total       0.89      0.65      0.73      0.73      0.69      0.47       324

